# Homework2

Team: AiCoders
Members:
* Alessio_Parato
* Jan31415
* *Vel*


Task: The goal is to design and implement forecasting models to learn how to exploit past observations in the input sequences to correctly predict the future.


## Connection and Imports

### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/Homework2/'
%cd /content/drive/MyDrive/Colab Notebooks/Homework2

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/Homework2


### Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.model_selection import train_test_split

2.15.0


In [ ]:
def save_model(model, model_name, path, mse=None, val_mse=None, mae=None, val_mae=None, plot=False):
  #
  #the functions saves the model and the history of a fit
  #
  if plot:
    tfk.utils.plot_model(model, to_file=path+'/model_pic.png', show_shapes=True, expand_nested=True)
  model.save(path+'/'+model_name)
  pd.DataFrame(mse).to_csv(path+'/'+model_name+'/mse.csv')
  pd.DataFrame(mae).to_csv(path+'/'+model_name+'/mae.csv')
  if val_mse!=None and val_mae!=None:
    pd.DataFrame(val_mse).to_csv(path+'/'+model_name+'/val_mse.csv')
    pd.DataFrame(val_mae).to_csv(path+'/'+model_name+'/val_mae.csv')

def load_model(path):
  #
  #the functions loads the model and the history of a fit
  #
  model = tfk.models.load_model(path)
  mse = pd.read_csv(path+'/mse.csv')
  val_mse = pd.read_csv(path+'/val_mse.csv')
  mae = pd.read_csv(path+'/mae.csv')
  val_mae = pd.read_csv(path+'/val_mae.csv')

  return (model, mse, val_mse, mae, val_mae)

def vali(cate_name, x, y):
  model1 = load_model('Cate_RESNET/model1'+cate_name)[0]
  model2 = load_model('Cate_RESNET/model2'+cate_name)[0]
  model3 = load_model('Cate_RESNET/model3'+cate_name)[0]
  print('Cate '+cate_name+':')
  print(f'1:{model1.evaluate(x, y, verbose=0)}')
  print(f'2:{model2.evaluate(x, y, verbose=0)}')
  print(f'3:{model3.evaluate(x, y, verbose=0)}')

## Load data

* 'training_data.npy': numpy array of shape (48000, 2776). 48000 time series of length 2776.
* 'valid_periods.npy': numpy array of type (48000, 2), containing for each of the time series the start and end index of the current series, i.e. the part without padding.
* 'categories.npy': numpy array of shape (48000,), containing for each of the time series the code of its category. The possible categories are in {'A', 'B', 'C', 'D', 'E', 'F'}.

In [ ]:
#loads the unzipped dataset and divides it into data and labels
t_d = np.load('Data/training_data.npy')
cate = np.load('Data/categories.npy')
v_p = np.load('Data/valid_periods.npy')
len_v_p = v_p[:,1]-v_p[:,0]
# Print shapes of the datasets
print(f"training shape: {t_d.shape}")
print(f"categories shape: {cate.shape}")
print(f"valid_periods shape: {v_p.shape}")

training shape: (48000, 2776)
categories shape: (48000,)
valid_periods shape: (48000, 2)


### Process the dataset and Split in train, validation and test

In [ ]:
A = np.reshape((np.where(cate=='A')), 5728)
A_train, A_v_p, A_len_v_p = t_d[A], v_p[A], len_v_p[A]
B = np.reshape((np.where(cate=='B')), 10987)
B_train, B_v_p, B_len_v_p = t_d[B], v_p[B], len_v_p[B]
C = np.reshape((np.where(cate=='C')), 10017)
C_train, C_v_p, C_len_v_p = t_d[C], v_p[C], len_v_p[C]
D = np.reshape((np.where(cate=='D')), 10016)
D_train, D_v_p, D_len_v_p = t_d[D], v_p[D], len_v_p[D]
E = np.reshape((np.where(cate=='E')), 10975)
E_train, E_v_p, E_len_v_p = t_d[E], v_p[E], len_v_p[E]
F = np.reshape((np.where(cate=='F')), 277)
F_train, F_v_p, F_len_v_p = t_d[F], v_p[F], len_v_p[F]

In [ ]:
def build_sequences(data, valid_periods, window=200, telescope=9, step=209):
    dataset = []
    labels = []
    da = data.copy()
    va_p = valid_periods.copy()

    for i in range(da.shape[0]):
      series = da[i,va_p[i,0]:va_p[i,1]]
      idx = 0
      while(idx+window+telescope < series.shape[0]):
        dataset.append(series[idx:idx+window])
        labels.append(series[idx+window:idx+window+telescope])
        idx = idx+step

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels
def build_sequences2(data, window=200, telescope=9, step=209):
    dataset = []
    labels = []
    da = data.copy()

    for series in da:
      idx = 0
      while(idx+window+telescope < series.shape[0]):
        dataset.append(series[idx:idx+window])
        labels.append(series[idx+window:idx+window+telescope])
        idx = idx+step

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
A_x, A_y = build_sequences(t_d[A], v_p[A], telescope=18, step=77)
print(f'Number of Training: {A_x.shape[0]}')
B_x, B_y = build_sequences(t_d[B], v_p[B],telescope=18, step=77)
print(f'Number of Training: {B_x.shape[0]}')
C_x, C_y = build_sequences(t_d[C], v_p[C],telescope=18, step=77)
print(f'Number of Training: {C_x.shape[0]}')
D_x, D_y = build_sequences(t_d[D], v_p[D],telescope=18, step=77)
print(f'Number of Training: {D_x.shape[0]}')
E_x, E_y = build_sequences(t_d[E], v_p[E],telescope=18, step=77)
print(f'Number of Training: {E_x.shape[0]}')
F_x, F_y = build_sequences(t_d[F], v_p[F],telescope=18, step=77)
print(f'Number of Training: {F_x.shape[0]}')

Number of Training: 6765
Number of Training: 5018
Number of Training: 7630
Number of Training: 9801
Number of Training: 6296
Number of Training: 196


In [ ]:
test_size = 0.2
Ax_t_v, Ax_t, Ay_t_v, Ay_t = train_test_split(A_x, A_y, random_state=seed, test_size=test_size)
Bx_t_v, Bx_t, By_t_v, By_t = train_test_split(B_x, B_y, random_state=seed, test_size=test_size)
Cx_t_v, Cx_t, Cy_t_v, Cy_t = train_test_split(C_x, C_y, random_state=seed, test_size=test_size)
Dx_t_v, Dx_t, Dy_t_v, Dy_t = train_test_split(D_x, D_y, random_state=seed, test_size=test_size)
Ex_t_v, Ex_t, Ey_t_v, Ey_t = train_test_split(E_x, E_y, random_state=seed, test_size=test_size)
Fx_t_v, Fx_t, Fy_t_v, Fy_t = train_test_split(F_x, F_y, random_state=seed, test_size=test_size)

In [ ]:
t_list = []
for i in range(t_d.shape[0]):
  t_list.append(t_d[i,v_p[i,0]:])

In [ ]:
tx, ty = build_sequences2(t_list, window=200, telescope=18, step=200)
print(tx.shape)

(23016, 200)


# Plan 1

In this Section we tested a ResNet-like CNN network with eight Conv1D layers

We have again trained the network first with all categories simultaneously and then with each category individually to specialise the network for each category

In [ ]:
def simple_model_RESNET():
    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=(200,1), name='input_layer')

    x = tfkl.Conv1D(16, 10, padding='same', activation='relu', name='conv1')(input_layer)
    x = tfkl.Conv1D(32, 10, padding='same', activation='relu', name='conv2')(x)
    x1 = input_layer + x
    x2 = tfkl.AveragePooling1D(3, name='avg1')(x1)
    x = tfkl.Conv1D(32, 10, padding='same', activation='relu', name='conv3')(x2)
    x = tfkl.Conv1D(32, 10, padding='same', activation='relu', name='conv4')(x)
    x = x + x2
    x = tfkl.Conv1D(64, 20, padding='same', activation='relu', name='conv5')(x)
    x3= tfkl.AveragePooling1D(4, name='avg2')(x)
    x = tfkl.Conv1D(64, 10, padding='same', activation='relu', name='conv6')(x3)
    x = tfkl.Conv1D(64, 10, padding='same', activation='relu', name='conv7')(x)
    x = x3 +x
    x = tfkl.Conv1D(128, 20, padding='same', activation='relu', name='conv8')(x)

    gap = tfkl.GlobalAveragePooling1D(name='gap')(x)

    output_layer = tfkl.Dense(18, activation='linear', name='output')(gap)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='simp_model')
    return model

In [ ]:
model = simple_model_RESNET()
model.summary()

Model: "simp_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 200, 1)]             0         []                            
                                                                                                  
 conv1 (Conv1D)              (None, 200, 16)              176       ['input_layer[0][0]']         
                                                                                                  
 conv2 (Conv1D)              (None, 200, 32)              5152      ['conv1[0][0]']               
                                                                                                  
 tf.__operators__.add_30 (T  (None, 200, 32)              0         ['input_layer[0][0]',         
 FOpLambda)                                                          'conv2[0][0]']      

In [ ]:
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
).history
save_model(model, 'model1', 'Cate_RESNET', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model
model = load_model('Cate_RESNET/model1')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
).history
save_model(model, 'model2', 'Cate_RESNET', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model
model = load_model('Cate_RESNET/model2')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])
tx, ty = build_sequences2(t_list, window=200, telescope=18, step=150)
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
).history
save_model(model, 'model3', 'Cate_RESNET', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Epoch 1/200
162/162 [==============================] - 38s 219ms/step - loss: 0.0239 - mse: 0.0239 - mae: 0.1082 - val_loss: 0.0169 - val_mse: 0.0169 - val_mae: 0.0922 - lr: 0.0010
Epoch 2/200
162/162 [==============================] - 35s 215ms/step - loss: 0.0131 - mse: 0.0131 - mae: 0.0810 - val_loss: 0.0152 - val_mse: 0.0152 - val_mae: 0.0849 - lr: 0.0010
Epoch 3/200
162/162 [==============================] - 36s 220ms/step - loss: 0.0124 - mse: 0.0124 - mae: 0.0783 - val_loss: 0.0145 - val_mse: 0.0145 - val_mae: 0.0817 - lr: 0.0010
Epoch 4/200
162/162 [==============================] - 37s 229ms/step - loss: 0.0118 - mse: 0.0118 - mae: 0.0757 - val_loss: 0.0141 - val_mse: 0.0141 - val_mae: 0.0801 - lr: 0.0010
Epoch 5/200
162/162 [==============================] - 35s 215ms/step - loss: 0.0113 - mse: 0.0113 - mae: 0.0738 - val_loss: 0.0138 - val_mse: 0.0138 - val_mae: 0.0788 - lr: 0.0010
Epoch 6/200
162/162 [==============================] - 36s 220ms/step - loss: 0.0111 - mse: 0.0

## Training function

In [ ]:
def training(folder, cate, cate_Char, step1, step2, test_size = 0.2):
    x, y = build_sequences(t_d[cate], v_p[cate], window=200, telescope=18, step = step1)
    print(f'Number of Training: {x.shape[0]}')

    x_t_v, x_t, y_t_v, y_t = train_test_split(x, y, random_state=seed, test_size=test_size)
    model = load_model(folder+'/model3')[0]
    model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(
      x = x_t_v,
      y = y_t_v,
      batch_size = 128, epochs = 200, validation_split=.1,
      callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
    ).history
    save_model(model, 'model1_'+cate_Char, folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
    del model
    model = load_model(folder+'/model1_'+cate_Char)[0]
    model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(
      x = x_t_v,
      y = y_t_v,
      batch_size = 64, epochs = 200, validation_split=.1,
      callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
    ).history
    save_model(model, 'model2_'+cate_Char, folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
    del model

    model = load_model(folder+'/model2_'+cate_Char)[0]
    model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])
    x, y = build_sequences(t_d[cate], v_p[cate], window=200, telescope=18, step = step2)
    print(f'Number of Training: {x.shape[0]}')
    x_t_v, x_t, y_t_v, y_t = train_test_split(x, y, random_state=seed, test_size=test_size)

    history = model.fit(
      x = x_t_v,
      y = y_t_v,
      batch_size = 64, epochs = 200, validation_split=.1,
      callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)]
    ).history
    save_model(model, 'model3_'+cate_Char, folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

## Cate A

In [ ]:
training('Cate_RESNET', A, 'A', step1=150, step2=90, test_size = 0.2)

Number of Training: 5062
Epoch 1/200
29/29 [==============================] - 10s 236ms/step - loss: 0.0030 - mse: 0.0030 - mae: 0.0405 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0338 - lr: 0.0010
Epoch 2/200
29/29 [==============================] - 6s 190ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0321 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0326 - lr: 0.0010
Epoch 3/200
29/29 [==============================] - 8s 271ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0298 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0322 - lr: 0.0010
Epoch 4/200
29/29 [==============================] - 6s 190ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0284 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0325 - lr: 0.0010
Epoch 5/200
29/29 [==============================] - 7s 256ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0273 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0326 - lr: 0.0010
Epoch 6/200
29/29 [==============================] - 6s 200ms/step - loss: 0.0014 - 

## Cate B

In [ ]:
training('Cate_RESNET', B, 'B', step1=80, step2=60, test_size = 0.2)

Number of Training: 4944
Epoch 1/200
28/28 [==============================] - 10s 282ms/step - loss: 0.0112 - mse: 0.0112 - mae: 0.0675 - val_loss: 0.0094 - val_mse: 0.0094 - val_mae: 0.0637 - lr: 0.0010
Epoch 2/200
28/28 [==============================] - 5s 194ms/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0617 - val_loss: 0.0091 - val_mse: 0.0091 - val_mae: 0.0630 - lr: 0.0010
Epoch 3/200
28/28 [==============================] - 8s 275ms/step - loss: 0.0075 - mse: 0.0075 - mae: 0.0560 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0590 - lr: 0.0010
Epoch 4/200
28/28 [==============================] - 5s 188ms/step - loss: 0.0064 - mse: 0.0064 - mae: 0.0517 - val_loss: 0.0082 - val_mse: 0.0082 - val_mae: 0.0586 - lr: 0.0010
Epoch 5/200
28/28 [==============================] - 6s 209ms/step - loss: 0.0056 - mse: 0.0056 - mae: 0.0491 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0582 - lr: 0.0010
Epoch 6/200
28/28 [==============================] - 7s 266ms/step - loss: 0.0050 - 

## Cate C

In [ ]:
training('Cate_RESNET', C, 'C', step1=150, step2=100, test_size = 0.2)

Number of Training: 5201
Epoch 1/200
30/30 [==============================] - 9s 223ms/step - loss: 0.0033 - mse: 0.0033 - mae: 0.0413 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0348 - lr: 0.0010
Epoch 2/200
30/30 [==============================] - 7s 248ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0327 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0329 - lr: 0.0010
Epoch 3/200
30/30 [==============================] - 6s 196ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0308 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0328 - lr: 0.0010
Epoch 4/200
30/30 [==============================] - 8s 268ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0292 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0339 - lr: 0.0010
Epoch 5/200
30/30 [==============================] - 6s 186ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0278 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0336 - lr: 0.0010
Epoch 6/200
30/30 [==============================] - 7s 247ms/step - loss: 0.0015 - m

## Cate D

In [ ]:
training('Cate_RESNET', D, 'D', step1=150, step2=100, test_size = 0.2)

Number of Training: 6777
Epoch 1/200
39/39 [==============================] - 12s 233ms/step - loss: 0.0041 - mse: 0.0041 - mae: 0.0455 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0352 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 10s 258ms/step - loss: 0.0024 - mse: 0.0024 - mae: 0.0339 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0344 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 7s 192ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0326 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0335 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 10s 251ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0315 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0345 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 7s 192ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0315 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0347 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 10s 254ms/step - loss: 0.0018

## Cate E

In [ ]:
training('Cate_RESNET', E, 'E', step1=150, step2=100, test_size = 0.2)

Number of Training: 4630
Epoch 1/200
27/27 [==============================] - 9s 261ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0408 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0332 - lr: 0.0010
Epoch 2/200
27/27 [==============================] - 5s 184ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0304 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0317 - lr: 0.0010
Epoch 3/200
27/27 [==============================] - 6s 237ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0287 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0324 - lr: 0.0010
Epoch 4/200
27/27 [==============================] - 6s 209ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0267 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0306 - lr: 0.0010
Epoch 5/200
27/27 [==============================] - 5s 184ms/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0253 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0309 - lr: 0.0010
Epoch 6/200
27/27 [==============================] - 7s 270ms/step - loss: 0.0012 - m

## Cate F

In [ ]:
training('Cate_RESNET', F, 'F', step1=10, step2=5, test_size = 0.2)

Number of Training: 1245
Epoch 1/200
7/7 [==============================] - 5s 347ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0533 - val_loss: 0.0093 - val_mse: 0.0093 - val_mae: 0.0485 - lr: 0.0010
Epoch 2/200
7/7 [==============================] - 1s 195ms/step - loss: 0.0050 - mse: 0.0050 - mae: 0.0374 - val_loss: 0.0087 - val_mse: 0.0087 - val_mae: 0.0455 - lr: 0.0010
Epoch 3/200
7/7 [==============================] - 1s 195ms/step - loss: 0.0040 - mse: 0.0040 - mae: 0.0334 - val_loss: 0.0086 - val_mse: 0.0086 - val_mae: 0.0434 - lr: 0.0010
Epoch 4/200
7/7 [==============================] - 1s 194ms/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0299 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0401 - lr: 0.0010
Epoch 5/200
7/7 [==============================] - 1s 194ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0292 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0426 - lr: 0.0010
Epoch 6/200
7/7 [==============================] - 1s 192ms/step - loss: 0.0030 - mse: 0.0030 -

## Validation

In [ ]:
vali('', Ax_t, Ay_t)
vali('_A', Ax_t, Ay_t)
vali('', Bx_t, By_t)
vali('_B', Bx_t, By_t)
vali('', Cx_t, Cy_t)
vali('_C', Cx_t, Cy_t)
vali('', Dx_t, Dy_t)
vali('_D', Dx_t, Dy_t)
vali('', Ex_t, Ey_t)
vali('_E', Ex_t, Ey_t)
vali('', Fx_t, Fy_t)
vali('_F', Fx_t, Fy_t)

Cate :
1:[0.006507514975965023, 0.006507514975965023, 0.056555382907390594]
2:[0.005154535174369812, 0.005154535174369812, 0.04959241673350334]
3:[0.004208726342767477, 0.004208726342767477, 0.04377391189336777]
Cate _A:
1:[0.0040392447263002396, 0.0040392447263002396, 0.04109322279691696]
2:[0.0039107236079871655, 0.0039107236079871655, 0.039244696497917175]
3:[0.004005896858870983, 0.004005896858870983, 0.04016470909118652]
Cate :
1:[0.01181472185999155, 0.01181472185999155, 0.06758109480142593]
2:[0.011771212331950665, 0.011771212331950665, 0.06693819910287857]
3:[0.011461412534117699, 0.011461412534117699, 0.06450365483760834]
Cate _B:
1:[0.006761952769011259, 0.006761952769011259, 0.050365276634693146]
2:[0.006286618299782276, 0.006286618299782276, 0.0459802970290184]
3:[0.006957764271646738, 0.006957764271646738, 0.04936068505048752]
Cate :
1:[0.008314521051943302, 0.008314521051943302, 0.060933370143175125]
2:[0.006964830216020346, 0.006964830216020346, 0.055323243141174316]
3:[

# Plan 2

Now we added in the beginning two dense layers the last one with the sigmoid activation function and multiplied the output with the original input layer to get a weighted input for the network. We wanted to recreate the attention layer in a very simple way.
We also added an extra new 'path' by adding from the original input a Conv1D layer and one dense layer. At the end we calculated the average of both outputs.

## Training function

In [ ]:
def Training_Signal(folder, cate, cate_Char, step1, test_size = 0.2):
    model_old = load_model(folder+'/model3_'+cate_Char)[0]

    input_layer = tfkl.Input(shape=200, name='input_layer')

    d1 = tfkl.Dense(200, activation='relu', name='Dense1')(input_layer)
    signal = tfkl.Dense(200, activation='sigmoid', name='Signal')(d1)
    mult = tfkl.Multiply()([signal, input_layer])
    out1 = model_old(mult)

    expand = tf.expand_dims(input_layer, axis=2)
    conv1 = tfkl.Conv1D(200, 200, padding='valid', activation='relu', name='conv1_ex')(expand)
    gap = tfkl.GlobalAveragePooling1D(name='gap')(conv1)
    out2 = model_old(gap)
    avg = tfkl.Average()([out1, out2])

    model = tfk.Model(inputs=input_layer, outputs=avg, name='model')

    model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(
      x = tx,
      y = ty,
      batch_size = 128, epochs = 30, validation_split=.1,
      verbose=2
    ).history
    save_model(model, 'model1_'+cate_Char+'_Signal', folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
    del model

    x, y = build_sequences(t_d[cate], v_p[cate], window=200, telescope=18, step = step1)
    print(f'Number of Training: {x.shape[0]}')
    x_t_v, x_t, y_t_v, y_t = train_test_split(x, y, random_state=seed, test_size=test_size)

    model = load_model(folder+'/model1_'+cate_Char+'_Signal')[0]
    model.compile(loss='mae', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(
      x = x_t_v,
      y = y_t_v,
      batch_size = 64, epochs = 200, validation_split=.1,
      callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)],
      verbose=2
    ).history
    save_model(model, 'model2_'+cate_Char+'_Signal', folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
    del model

    model = load_model(folder+'/model2_'+cate_Char+'_Signal')[0]
    model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(
      x = x_t_v,
      y = y_t_v,
      batch_size = 64, epochs = 200, validation_split=.1,
      callbacks = [ tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
                  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)],
      verbose=2
    ).history
    save_model(model, 'model3_'+cate_Char+'_Signal', folder, history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)


## Cate A

In [ ]:
Training_Signal('Cate_RESNET', A, 'A', step1=90, test_size = 0.2)

Epoch 1/30
162/162 - 92s - loss: 0.0091 - mse: 0.0091 - mae: 0.0645 - val_loss: 0.0115 - val_mse: 0.0115 - val_mae: 0.0706 - 92s/epoch - 567ms/step
Epoch 2/30
162/162 - 85s - loss: 0.0041 - mse: 0.0041 - mae: 0.0447 - val_loss: 0.0122 - val_mse: 0.0122 - val_mae: 0.0725 - 85s/epoch - 525ms/step
Epoch 3/30
162/162 - 86s - loss: 0.0034 - mse: 0.0034 - mae: 0.0408 - val_loss: 0.0119 - val_mse: 0.0119 - val_mae: 0.0709 - 86s/epoch - 530ms/step
Epoch 4/30
162/162 - 85s - loss: 0.0030 - mse: 0.0030 - mae: 0.0383 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0700 - 85s/epoch - 528ms/step
Epoch 5/30
162/162 - 84s - loss: 0.0027 - mse: 0.0027 - mae: 0.0367 - val_loss: 0.0121 - val_mse: 0.0121 - val_mae: 0.0711 - 84s/epoch - 519ms/step
Epoch 6/30
162/162 - 87s - loss: 0.0025 - mse: 0.0025 - mae: 0.0351 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0693 - 87s/epoch - 535ms/step
Epoch 7/30
162/162 - 85s - loss: 0.0024 - mse: 0.0024 - mae: 0.0344 - val_loss: 0.0117 - val_mse: 0.0117 - val_m

## Cate B

In [ ]:
Training_Signal('Cate_RESNET', B, 'B', step1=60, test_size = 0.2)

Epoch 1/30
162/162 - 88s - loss: 0.0099 - mse: 0.0099 - mae: 0.0677 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0597 - 88s/epoch - 542ms/step
Epoch 2/30
162/162 - 84s - loss: 0.0049 - mse: 0.0049 - mae: 0.0489 - val_loss: 0.0073 - val_mse: 0.0073 - val_mae: 0.0565 - 84s/epoch - 519ms/step
Epoch 3/30
162/162 - 80s - loss: 0.0039 - mse: 0.0039 - mae: 0.0436 - val_loss: 0.0074 - val_mse: 0.0074 - val_mae: 0.0563 - 80s/epoch - 492ms/step
Epoch 4/30
162/162 - 79s - loss: 0.0033 - mse: 0.0033 - mae: 0.0402 - val_loss: 0.0074 - val_mse: 0.0074 - val_mae: 0.0562 - 79s/epoch - 490ms/step
Epoch 5/30
162/162 - 81s - loss: 0.0030 - mse: 0.0030 - mae: 0.0381 - val_loss: 0.0076 - val_mse: 0.0076 - val_mae: 0.0571 - 81s/epoch - 501ms/step
Epoch 6/30
162/162 - 88s - loss: 0.0027 - mse: 0.0027 - mae: 0.0364 - val_loss: 0.0073 - val_mse: 0.0073 - val_mae: 0.0553 - 88s/epoch - 540ms/step
Epoch 7/30
162/162 - 83s - loss: 0.0025 - mse: 0.0025 - mae: 0.0355 - val_loss: 0.0077 - val_mse: 0.0077 - val_m

## Cate C

In [ ]:
Training_Signal('Cate_RESNET', C, 'C', step1=100, test_size = 0.2)

Epoch 1/30
162/162 - 89s - loss: 0.0086 - mse: 0.0086 - mae: 0.0625 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0705 - 89s/epoch - 551ms/step
Epoch 2/30
162/162 - 88s - loss: 0.0037 - mse: 0.0037 - mae: 0.0424 - val_loss: 0.0119 - val_mse: 0.0119 - val_mae: 0.0704 - 88s/epoch - 544ms/step
Epoch 3/30
162/162 - 83s - loss: 0.0032 - mse: 0.0032 - mae: 0.0394 - val_loss: 0.0120 - val_mse: 0.0120 - val_mae: 0.0709 - 83s/epoch - 515ms/step
Epoch 4/30
162/162 - 85s - loss: 0.0028 - mse: 0.0028 - mae: 0.0368 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0695 - 85s/epoch - 528ms/step
Epoch 5/30
162/162 - 84s - loss: 0.0025 - mse: 0.0025 - mae: 0.0351 - val_loss: 0.0120 - val_mse: 0.0120 - val_mae: 0.0708 - 84s/epoch - 520ms/step
Epoch 6/30
162/162 - 84s - loss: 0.0024 - mse: 0.0024 - mae: 0.0341 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0692 - 84s/epoch - 521ms/step
Epoch 7/30
162/162 - 82s - loss: 0.0022 - mse: 0.0022 - mae: 0.0332 - val_loss: 0.0120 - val_mse: 0.0120 - val_m

## Cate D

In [ ]:
Training_Signal('Cate_RESNET', D, 'D', step1=100, test_size = 0.2)

Epoch 1/30
162/162 - 83s - loss: 0.0086 - mse: 0.0086 - mae: 0.0619 - val_loss: 0.0119 - val_mse: 0.0119 - val_mae: 0.0711 - 83s/epoch - 509ms/step
Epoch 2/30
162/162 - 76s - loss: 0.0037 - mse: 0.0037 - mae: 0.0423 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0701 - 76s/epoch - 470ms/step
Epoch 3/30
162/162 - 79s - loss: 0.0031 - mse: 0.0031 - mae: 0.0392 - val_loss: 0.0121 - val_mse: 0.0121 - val_mae: 0.0726 - 79s/epoch - 487ms/step
Epoch 4/30
162/162 - 77s - loss: 0.0028 - mse: 0.0028 - mae: 0.0369 - val_loss: 0.0120 - val_mse: 0.0120 - val_mae: 0.0711 - 77s/epoch - 474ms/step
Epoch 5/30
162/162 - 76s - loss: 0.0026 - mse: 0.0026 - mae: 0.0356 - val_loss: 0.0124 - val_mse: 0.0124 - val_mae: 0.0721 - 76s/epoch - 471ms/step
Epoch 6/30
162/162 - 82s - loss: 0.0024 - mse: 0.0024 - mae: 0.0344 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0688 - 82s/epoch - 504ms/step
Epoch 7/30
162/162 - 75s - loss: 0.0022 - mse: 0.0022 - mae: 0.0331 - val_loss: 0.0121 - val_mse: 0.0121 - val_m

## Cate E

In [ ]:
Training_Signal('Cate_RESNET', E, 'E', step1=100, test_size = 0.2)

Epoch 1/30
162/162 - 83s - loss: 0.0084 - mse: 0.0084 - mae: 0.0616 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0707 - 83s/epoch - 510ms/step
Epoch 2/30
162/162 - 80s - loss: 0.0037 - mse: 0.0037 - mae: 0.0427 - val_loss: 0.0118 - val_mse: 0.0118 - val_mae: 0.0699 - 80s/epoch - 496ms/step
Epoch 3/30
162/162 - 76s - loss: 0.0032 - mse: 0.0032 - mae: 0.0392 - val_loss: 0.0122 - val_mse: 0.0122 - val_mae: 0.0731 - 76s/epoch - 470ms/step
Epoch 4/30
162/162 - 78s - loss: 0.0028 - mse: 0.0028 - mae: 0.0367 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0698 - 78s/epoch - 481ms/step
Epoch 5/30
162/162 - 80s - loss: 0.0025 - mse: 0.0025 - mae: 0.0352 - val_loss: 0.0118 - val_mse: 0.0118 - val_mae: 0.0698 - 80s/epoch - 492ms/step
Epoch 6/30
162/162 - 80s - loss: 0.0025 - mse: 0.0025 - mae: 0.0346 - val_loss: 0.0115 - val_mse: 0.0115 - val_mae: 0.0684 - 80s/epoch - 491ms/step
Epoch 7/30
162/162 - 79s - loss: 0.0023 - mse: 0.0023 - mae: 0.0334 - val_loss: 0.0120 - val_mse: 0.0120 - val_m

## Cate F

In [ ]:
Training_Signal('Cate_RESNET', F, 'F', step1=5, test_size = 0.2)

Epoch 1/30
162/162 - 81s - loss: 0.0098 - mse: 0.0098 - mae: 0.0676 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0709 - 81s/epoch - 497ms/step
Epoch 2/30
162/162 - 79s - loss: 0.0045 - mse: 0.0045 - mae: 0.0467 - val_loss: 0.0118 - val_mse: 0.0118 - val_mae: 0.0712 - 79s/epoch - 485ms/step
Epoch 3/30
162/162 - 79s - loss: 0.0036 - mse: 0.0036 - mae: 0.0419 - val_loss: 0.0121 - val_mse: 0.0121 - val_mae: 0.0728 - 79s/epoch - 485ms/step
Epoch 4/30
162/162 - 75s - loss: 0.0031 - mse: 0.0031 - mae: 0.0388 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0698 - 75s/epoch - 465ms/step
Epoch 5/30
162/162 - 79s - loss: 0.0028 - mse: 0.0028 - mae: 0.0371 - val_loss: 0.0117 - val_mse: 0.0117 - val_mae: 0.0693 - 79s/epoch - 486ms/step
Epoch 6/30
162/162 - 81s - loss: 0.0026 - mse: 0.0026 - mae: 0.0358 - val_loss: 0.0114 - val_mse: 0.0114 - val_mae: 0.0683 - 81s/epoch - 502ms/step
Epoch 7/30
162/162 - 76s - loss: 0.0025 - mse: 0.0025 - mae: 0.0349 - val_loss: 0.0117 - val_mse: 0.0117 - val_m

## Validation

In [ ]:
vali('_A', Ax_t, Ay_t)
vali('_A_Signal', Ax_t, Ay_t)
vali('_B', Bx_t, By_t)
vali('_B_Signal', Bx_t, By_t)
vali('_C', Cx_t, Cy_t)
vali('_C_Signal', Cx_t, Cy_t)
vali('_D', Dx_t, Dy_t)
vali('_D_Signal', Dx_t, Dy_t)
vali('_E', Ex_t, Ey_t)
vali('_E_Signal', Ex_t, Ey_t)
vali('_F', Fx_t, Fy_t)
vali('_F_Signal', Fx_t, Fy_t)

Cate _A:
1:[0.0040392447263002396, 0.0040392447263002396, 0.04109322279691696]
2:[0.0039107236079871655, 0.0039107236079871655, 0.039244696497917175]
3:[0.004005896858870983, 0.004005896858870983, 0.04016470909118652]
Cate _A_Signal:
1:[0.0038571131881326437, 0.0038571131881326437, 0.04065021872520447]
2:[0.035206958651542664, 0.0035888301208615303, 0.035206958651542664]
3:[0.0037102142814546824, 0.0037102142814546824, 0.036938346922397614]
Cate _B:
1:[0.006761952769011259, 0.006761952769011259, 0.050365276634693146]
2:[0.006286618299782276, 0.006286618299782276, 0.0459802970290184]
3:[0.006957764271646738, 0.006957764271646738, 0.04936068505048752]
Cate _B_Signal:
1:[0.010161672718822956, 0.010161672718822956, 0.06022505834698677]
2:[0.04330410063266754, 0.006958136335015297, 0.04330410063266754]
3:[0.007050007581710815, 0.007050007581710815, 0.04486485570669174]
Cate _C:
1:[0.00590028241276741, 0.00590028241276741, 0.04891431704163551]
2:[0.005720566492527723, 0.005720566492527723, 0